## Feature Engineering

El proceso realizado aquí se centra en calcular el sentimiento de cada revisión (review) para luego asignarle una puntuación de análisis de sentimientos (sentiment_analysis) en función de esa evaluación y de si se recomienda (recommend) o no.\
Antes de calcular el análisis de sentimientos se aplica el stemming (PorterStemmer) definiendo la función apply_stemming para aplicar el stemming a la columna 'review' del DataFrame. Esto reduce las palabras a su forma base para capturar mejor el significado general y reducir la dimensionalidad del texto.

*Importamos las librerías necesarias para realizar el análisis.*

In [7]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import PorterStemmer  # Importar el stemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer  # Importar el lematizador
import nltk

In [8]:
nltk.download('vader_lexicon')  # Descargar el modelo vader_lexicon de NLTK
nltk.download('punkt')  # Descargar el tokenizer de NLTK
nltk.download('wordnet')  # Descargar WordNet para lematización

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\beatr\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\beatr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\beatr\AppData\Roaming\nltk_data...


True

Cargar el dataset:\
*Cargamos el dataset en un DataFrame de pandas para poder manipularlo y realizar el análisis de sentimiento.*

In [9]:
df = pd.read_csv(r'C:\Users\beatr\OneDrive\Escritorio\henry\Proyecto Steam\Consulta_Data\user_reviews_clean.csv')

*Hacemos una copia para realizar las operaciones, mientras resguardamos los datos en el archivo original.*

In [ ]:
df_copy = df.copy()
df_copy

*Realizamos una pequeña limpieza previa al análisis.*

In [11]:
# Eliminar filas con valores nulos en la columna 'review'
# df = df.dropna(subset=['review'])

# Reemplazar expresiones
df_copy['review'] = df_copy['review'].replace(['10/10', '100/100'], 'Excelent', regex=True)

# Transformar todos los datos a tipo string
df_copy['review'] = df_copy['review'].astype(str)

* Inicializar el Analizador de Intensidad de Sentimiento (Sentiment Intensity Analyzer):
* Inicializar el stemmer
* Inicializar el lematizador\
*Creamos una instancia para cada uno de los procesos.*

In [12]:
# Inicializar el analizador de intensidad de sentimiento de NLTK
sid = SentimentIntensityAnalyzer()

# Inicializar el stemmer
stemmer = PorterStemmer()

# Inicializar el lematizador
lemmatizer = WordNetLemmatizer()  

Aplicación de Stemming a la Columna 'review' del DataFrame:\
*En este código, utilizamos el algoritmo de Stemming de NLTK para reducir las palabras a su forma base en la columna 'review' de nuestro DataFrame.*

In [13]:
# Definir la Función de Aplicación del Stemming
def apply_stemming(text):
    words = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

# Aplicar la función de stemming a la columna 'review' del DataFrame
df_copy['review'] = df_copy['review'].apply(apply_stemming)

Aplicación de Lematización a la Columna 'review' del DataFrame:\
*En este código, la función utiliza word_tokenize para dividir el texto en palabras y luego aplica la lematización a cada palabra utilizando lemmatizer.lemmatize(word). Finalmente, se unen las palabras lematizadas de nuevo en una cadena utilizando ' '.join(lemmatized_words) y se asigna el resultado a la columna 'review' del DataFrame df_copy.*

In [14]:
# Definir la Función de Aplicación de la Lematización
def apply_lemmatization(text):
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

# Aplicar la función de lematización a la columna 'review' del DataFrame
df_copy['review'] = df_copy['review'].apply(apply_lemmatization)

Aplicación del análisis de sentimiento:\
*Creamos una función que tomará una reseña y aplicará el análisis de sentimiento, devolviendo el valor correspondiente según la escala especificada (malo='0', neutral='1' y positivo='2'). Si la reseña está ausente, la función devolverá '1'. Y luego la aplicamos al DataFrame*

In [15]:
# Definir la Función de Análisis de Sentimiento
def get_sentiment_score(text, recommend):
    if pd.isnull(text):
        return 1
    sentiment_score = sid.polarity_scores(text)['compound']
    if recommend:
        sentiment_score += 0.5
    else:
        sentiment_score -= 0.5
    if sentiment_score <= -0.05:
        return 0
    elif sentiment_score >= 0.05:
        return 2
    else:
        return 1

# Aplicar la función get_sentiment_score a la columna de reseñas (review) para crear la nueva columna 'sentiment_analysis'.
df_copy['sentiment_analysis'] = df_copy.apply(lambda row: get_sentiment_score(row['review'], row['recommend']), axis=1)

df_copy
df_copy.drop(columns=['review'], inplace=True)

In [16]:
df_copy

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,"Posted November 5, 2011.",1250.0,True,2
1,76561197970982479,"Posted July 15, 2011.",22200.0,True,2
2,76561197970982479,"Posted April 21, 2011.",43110.0,True,2
3,js41637,"Posted June 24, 2014.",251610.0,True,2
4,js41637,"Posted September 8, 2013.",227300.0,True,2
...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70.0,True,2
59329,76561198312638244,Posted July 8.,362890.0,True,2
59330,LydiaMorley,Posted July 3.,273110.0,True,2
59331,LydiaMorley,Posted July 20.,730.0,True,2


* *Analizamos cómo se correlacionan los sentimientos expresados en las reseñas con las recomendaciones dadas por los usuarios.*
* *Imprimimos en pantalla el conteo de las reseñas que muestran una inconsistencia entre el sentimiento expresado y la calificación dada por los usuarios, mostrando la cantidad de reseñas con sentimiento negativo y calificación positiva, así como la cantidad de reseñas con sentimiento positivo y calificación negativa.*

In [ ]:
# Reseñas con sentimiento positivo pero con una recomendación negativa.
positive_negative = df_copy[(df_copy['sentiment_analysis'] == 2) & (df_copy['recommend'] == False)]

# Reseñas con sentimiento negativo pero con una recomendación positiva.
negative_positive = df_copy[(df_copy['sentiment_analysis'] == 0) & (df_copy['recommend'] == True)]

# Conteo de reseñas con sentimiento negativo y recomendación positiva.
count_negative_positive = len(negative_positive)

# Conteo de reseñas con sentimiento positivo y recomendación negativa.
count_positive_negative = len(positive_negative)

print("Conteo de reseñas con sentimiento negativo y recomendación positiva:", count_negative_positive)
print("Conteo de reseñas con sentimiento positivo y recomendación negativa:", count_positive_negative)

Conteo de reseñas con sentimiento negativo y recomendación positiva: 2904
Conteo de reseñas con sentimiento positivo y recomendación negativa: 1323


*Calculamos el porcentaje de errores respecto de la cantidad total de registros para conocer el margen de error del análisis de sentimiento.*

In [ ]:
total_mistakes = 2904 + 1323
total_records = 59333
percent_mistakes = (total_mistakes / total_records) * 100

print("Porcentaje de errores:", percent_mistakes)

Porcentaje de errores: 7.124197326951275


*(Opcional) Guardamos el DataFrame actualizado en un nuevo archivo CSV.*

In [18]:
df_copy.to_parquet(r'C:\Users\beatr\OneDrive\Escritorio\henry\Proyecto Steam\DF_clean\Sentiment_Analysis_parquet', index=False)